In [1]:
%matplotlib widget

import os  
import sys  
sys.path.insert(0, os.getenv('HOME')+'/pycode/MscThesis/')
import pandas as pd
from amftrack.util.sys import get_dates_datetime, get_dirname, get_plate_number, get_postion_number

import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
from amftrack.pipeline.functions.node_id import orient
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.experiment_class_surf import Experiment, Edge, Node
from amftrack.pipeline.paths.directory import run_parallel, find_state, directory_scratch, directory_project
from amftrack.notebooks.analysis.util import *
from scipy import stats
from scipy.ndimage.filters import uniform_filter1d
from scipy import sparse
import numpy as np
import os
from datetime import datetime, timedelta
import pandas
path_code = "/home/cbisot/pycode/"
plate_info = pandas.read_excel(path_code + 'MscThesis/plate_info/SummaryAnalizedPlates.xlsx',engine='openpyxl',header=3,)

In [2]:
def get_Pside(plate_number):
    return(plate_info.loc[plate_info['Plate #'] == plate_number]['P-bait'])

In [89]:
get_Pside(395).values[0]

'right'

In [93]:
get_pos_nut((10,0,10),4,'P')

((34005, 11931), 'right')

In [11]:
directory = directory_project
exp = get_exp((28,0,14),directory)

begin = 2020-10-01 11:21:00 
  end = 2020-10-03 19:20:00


In [84]:
def get_length(exp,t):
    length=0
    for edge in exp.nx_graph[t].edges:
        edge_obj= Edge(Node(edge[0],exp),Node(edge[1],exp),exp)
        length+=get_length_um_edge(edge_obj,t)
    return(length)


In [85]:
[get_length(exp,t) for t in range(exp.ts)]

[1357017.2057098541,
 1393691.40573769,
 1441346.4998925133,
 1483465.249289077,
 1536511.450767385,
 1597264.2760047822,
 1664331.1186561834,
 1744229.4757101627,
 1832452.4571966603,
 1895890.044514469,
 1971489.3330531297,
 2033477.1671111505,
 2095788.7383510282,
 2160717.3692914154,
 2210800.518545975,
 2279689.888051225,
 2337126.380760785,
 2388667.3226391436,
 2455904.967060262,
 2494351.428872862,
 2604350.157458141,
 2641662.281776379,
 2682595.4417969994,
 2724403.0183652034,
 2776491.7694952977,
 2872333.535162379]

In [12]:
def get_raw(exp,t):
    date = exp.dates[t]
    directory_name = get_dirname(date,exp.plate)
    path_snap = exp.directory + directory_name
    im = read_mat(path_snap+'/Analysis/raw_image.mat')['raw']
    return(im)

In [16]:
def get_pos_baits(exp,t):
    raw_im = get_raw(exp,t)
    image = cv2.blur(raw_im,(100,100))
    output = image.copy()
    gray = image
    circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 2, 3000,minRadius = 200,maxRadius = 600,param1=10)
    # ensure at least some circles were found
    print('circles', circles)
    if circles is not None:
        # convert the (x, y) coordinates and radius of the circles to integers
        circles = np.round(circles[0, :]).astype("int")
    for (x, y, r) in circles[:2]:
        # draw the circle in the output image, then draw a rectangle
        # corresponding to the center of the circle
        cv2.circle(output, (x, y), r, (0, 255, 0), 4)
        cv2.rectangle(output, (x - 5, y - 5), (x + 5, y + 5), (0, 128, 255), -1)
        # show the output image
    fig = plt.figure(figsize=(10,9))
    ax = fig.add_subplot(111)
    ax.imshow(output)
    return(circles[:2])

In [56]:
def get_pos_baits_aligned(exp,t):
    pos_bait = get_pos_baits(exp,t)
    date = exp.dates[t]
    directory_name = get_dirname(date, exp.plate)
    path_snap = exp.directory + directory_name
    path_tile = path_snap + "/Img/TileConfiguration.txt.registered"
    skel = read_mat(path_snap + "/Analysis/skeleton_pruned_realigned.mat")
    Rot = skel["R"]
    trans = skel["t"]
    print(Rot,trans)
    real_pos=[]
    for x,y,r in pos_bait:
        compression = 5
        xs,ys = x*compression,y*compression
        rottrans = np.dot(Rot, np.array([ys, xs])) + trans
#         rottrans = np.array([xs, ys])
        xs, ys = round(rottrans[0]), round(rottrans[1])
        real_pos.append((xs,ys))
    pos_real = {}
    if real_pos[0][1]>=real_pos[1][1]:
        pos_real['right'] = real_pos[0]
        pos_real['left'] = real_pos[1]
    else:
        pos_real['right'] = real_pos[1]
        pos_real['left'] = real_pos[0]
    return(pos_real)
        

In [58]:
plt.close('all')
t=13
baits=get_pos_baits_aligned(exp,t)

circles [[[2031.  3085.   396.6]
  [7707.  3119.   400. ]
  [5179.    15.   457.8]]]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[[ 9.99999994e-01 -1.12628000e-04]
 [ 1.12628000e-04  9.99999994e-01]] [-102.01619074    9.10867534]


In [59]:
fig = plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.imshow(final_skel)
ax.scatter(baits['right'][1]//5,baits['right'][0]//5)
ax.scatter(baits['left'][1]//5,baits['left'][0]//5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [94]:
bait_positions={}
for treatment in ['25','baits']:
    insts = treatments[treatment]
    for inst in insts:
        bait_positions[inst] = pickle.load(open(f'{path_code}/MscThesis/Results/baits_{inst}.pick', "rb"))

In [65]:
bait_positions[(13,0,25)]

[{'right': (38575, 15505), 'left': (10155, 15335)},
 {'right': (38560, 15477), 'left': (10150, 15337)},
 {'right': (38565, 15489), 'left': (10165, 15333)},
 {'right': (38568, 15482), 'left': (10158, 15323)},
 {'right': (38565, 15467), 'left': (10154, 15321)},
 {'right': (38553, 15478), 'left': (10153, 15327)},
 {'right': (38571, 15476), 'left': (10171, 15337)},
 {'right': (38559, 15476), 'left': (10169, 15341)},
 {'right': (38559, 15492), 'left': (10169, 15318)},
 {'right': (38570, 15490), 'left': (10160, 15321)},
 {'right': (38559, 15490), 'left': (10159, 15336)},
 {'right': (38553, 15481), 'left': (10163, 15315)},
 {'right': (38571, 15473), 'left': (10161, 15328)},
 {'right': (38546, 15489), 'left': (10166, 15322)},
 {'right': (38557, 15485), 'left': (10157, 15315)},
 {'right': (38570, 15482), 'left': (10169, 15317)},
 {'right': (38563, 15468), 'left': (10153, 15324)},
 {'right': (38552, 15475), 'left': (10162, 15322)},
 {'right': (38544, 15468), 'left': (10154, 15324)},
 {'right': (

In [95]:
bait_positions_new={}
for treatment in ['25','baits']:
    insts = treatments[treatment]
    for inst in insts:
        bait_inst = bait_positions[inst]
        invert=[]
        for t in range(len(bait_inst)):
            pos_bait_right = bait_inst[t]['right']
            pos_bait_left = bait_inst[t]['left']
            invert.append({'right':(pos_bait_right[1],pos_bait_right[0]),'left':(pos_bait_left[1],pos_bait_left[0])})
        bait_positions_new[inst]=invert
        pickle.dump(invert,open(f'{path_code}/MscThesis/Results/baits2_{inst}.pick', "wb"))
bait_positions[(13,0,25)],bait_positions_new[(13,0,25)]

([{'right': (38575, 15505), 'left': (10155, 15335)},
  {'right': (38560, 15477), 'left': (10150, 15337)},
  {'right': (38565, 15489), 'left': (10165, 15333)},
  {'right': (38568, 15482), 'left': (10158, 15323)},
  {'right': (38565, 15467), 'left': (10154, 15321)},
  {'right': (38553, 15478), 'left': (10153, 15327)},
  {'right': (38571, 15476), 'left': (10171, 15337)},
  {'right': (38559, 15476), 'left': (10169, 15341)},
  {'right': (38559, 15492), 'left': (10169, 15318)},
  {'right': (38570, 15490), 'left': (10160, 15321)},
  {'right': (38559, 15490), 'left': (10159, 15336)},
  {'right': (38553, 15481), 'left': (10163, 15315)},
  {'right': (38571, 15473), 'left': (10161, 15328)},
  {'right': (38546, 15489), 'left': (10166, 15322)},
  {'right': (38557, 15485), 'left': (10157, 15315)},
  {'right': (38570, 15482), 'left': (10169, 15317)},
  {'right': (38563, 15468), 'left': (10153, 15324)},
  {'right': (38552, 15475), 'left': (10162, 15322)},
  {'right': (38544, 15468), 'left': (10154, 15

In [70]:
pickle.dump(bait_positions_new,open(f'{path_code}/MscThesis/Results/baits2_{inst}.pick', "wb"))

In [57]:
from amftrack.notebooks.analysis.data_info import *
inst_25 = [(13,0,25),(35,0,15),(29,0,20),(9,0,11),(9,13,35),(3,0,19),(37,0,8),(11,0,30),(19,0,25),(39,0,18)]
inst_bait = [(10,0,10), (14,0,11), (33,0,26),(4,2,18),(4,20,30),(39,117,137),(12,5,21),(28,0,14),(32,5,14),(32,15,44),(36,0,9),(40,0,14),(2,1,15),(2,17,35),(5,160,168),(11,158,164),(13,116,131)]
inst_30 = []
inst_25late = [(40,153,153+37),(39,269,329),(40,262,287)]
treatments = {'25*' : inst_25late,'25' : inst_25,'baits' : inst_bait, '30' : inst_30}
plate_number = {(9,0,11) : 296, (9,13,35) : 296, (3,0,19) : 340, (37,0,8) : 269,(11,0,30) : 314, (19,0,25) : 344, (13,0,25) : 298, (39,0,18) : 297, (35,0,15) : 351,(10,0,10) : 395,(14,0,11) : 399, (33,0,26) : 420, (4,2,18) : 423, (4,20,30) : 423,(8,0,17): 434 ,(8,20,30) : 434,(39,117,137) : 433, (12,5,21) : 436, (28,0,14): 405,(32,5,45):409,(36,0,9) : 419,(40,0,14) : 425,(2,1,15):435,(2,17,35):435,(5,160,168):382,(11,158,164) : 416,(13,116,131) : 424, (29,0,20) : 373,(32,15,44):409, (32,5,14) : 409, (40,153,153+37) : 69,(39,269,329) : 94, (40,262,287) : 102}
path = directory_project
pixel_conversion_factor = 1.725

def get_pos_nut(inst, t, nutrient):
    plate_num = plate_number[inst]
    side_P = get_Pside(plate_num).values[0]
    if nutrient == 'P':
        side_nut = side_P
    else:
        side_nut ='left' if side_P=='right' else 'left'
    if side_nut == 'none':
        side_nut = 'right'
    pos_nut = bait_positions[inst][t][side_nut]
    return(pos_nut,side_nut)

bait_positions={}
for treatment in ['25','baits']:
    insts = treatments[treatment]
    for inst in insts:
        bait_positions[inst] = pickle.load(open(f'{path_code}/MscThesis/Results/baits_{inst}.pick', "rb"))
        
for treatment in ['25']:
    insts = treatments[treatment]
    for inst in insts:
        print(inst)
        exp = get_exp(inst,path)
        skeletons = [sparse.csr_matrix(skel) for skel in exp.skeletons]
        RH, BAS, max_speeds, total_growths, widths_sp, lengths, branch_frequ,select_hyph = get_rh_bas(exp,criter)
        pos_baits = bait_positions[inst]
        for hyph in RH:
            for i,t in enumerate(hyph.ts[:-1]):
                pos_P, side_P = get_pos_nut(inst,t,'P')
                pos_N, side_N = get_pos_nut(inst,t,'N')
                pos = hyph.end.pos(t)
                dist_P = np.linalg.norm(np.array(pos)-np.array(pos_P))*pixel_conversion_factor
                print(pos_P,side_P,dist_P)
            break
        break

(13, 0, 25)
begin = 2020-07-01 15:57:00 
  end = 2020-07-05 23:57:00


KeyboardInterrupt: 

In [46]:
for hyph in RH:
    for i,t in enumerate(hyph.ts[:-1]):
        pos_P, side_P = get_pos_nut(inst,t,'P')
        pos_N, side_N = get_pos_nut(inst,t,'N')
        pos = hyph.end.pos(t)
        dist_P = np.linalg.norm(np.array(pos)-np.array(pos_P))*pixel_conversion_factor
        if (dist_P)<=40000:
            print(dist_P)
            break

39383.50253984846
38134.85184795694
37863.62838534
37478.33507749844
39254.893206962115
37257.55636952221
39091.763052240305
37421.00329555917
32646.219789165487
30704.642205780547
33718.81538897275
28899.95140465205
25691.877651470415
31347.669941520377
30346.600702930882
38314.170752167534
39566.33237018722
38458.836352862396
38807.900839173526
25556.800649790363
37777.52024314526
39997.573028528874


In [45]:
len(RH)

212

In [51]:
plt.close('all')
exp.plot([t],[[hyph.end.label]])

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [55]:
pos,pos_P

(array([12362, 33716]), (38542, 15469))

In [54]:
6700*5

33500

In [50]:
exp.load_compressed_skel()

In [40]:
np.linalg.norm(np.array(pos)-np.array(pos_N))*pixel_conversion_factor

11894.293420848924

In [52]:
exp.plot_raw(t)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
date = exp.dates[t]
directory_name = get_dirname(date,exp.plate)
path_snap = directory + directory_name
skel_info = read_mat(path_snap+'/Analysis/skeleton_realigned_compressed.mat')
skel = skel_info['skeleton']
#     skels.append(skel)
kernel = np.ones((5, 5), np.uint8)
itera=1
final_skel= cv2.dilate(skel.astype(np.uint8),kernel,iterations = itera)

In [27]:
exp.ts

16

In [7]:
import os  
import sys  
sys.path.insert(0, os.getenv('HOME')+'/pycode/MscThesis/')
sys.path.append( '/home/cbisot/pycode/MscThesis/amftrack/pipeline/functions')

from amftrack.notebooks.analysis.util import *
from amftrack.pipeline.paths.directory import path_code, directory_scratch, directory_project
from amftrack.notebooks.analysis.data_info import *
# for treatment in ['25','baits']:
#     insts = treatments[treatment]
#     for inst in insts:
#         exp = get_exp(inst,directory_project)
#         pos_baits_time = []
#         for t in range(exp.ts):
#             baits=get_pos_baits_aligned(exp,t)
#             pos_baits_time.append(baits)
#         pickle.dump(pos_baits_time, open(f'{path_code}/MscThesis/Results/baits_{inst}.pick', "wb"))

In [32]:
bait_positions={}
for treatment in ['25','baits']:
    insts = treatments[treatment]
    for inst in insts:
        bait_positions[inst] = pickle.load(open(f'{path_code}/MscThesis/Results/baits_{inst}.pick', "rb"))

In [63]:
bait_positions[35,0,15][0][]

{'right': (32925, 13085), 'left': (5325, 11825)}

In [ ]:
from amftrack.notebooks.analysis.util import *
from amftrack.pipeline.paths.directory import path_code, directory_scratch, directory_project
from amftrack.notebooks.analysis.data_info import *
window=800
for treatment in treatments.keys():
    insts = treatments[treatment]
    for inst in insts:
        pos_baits = bait_positions[inst]
        result = get_curvature_density(inst,window,directory_project,pos_baits)
        pickle.dump(result, open(f'{path_code}/MscThesis/Results/straight_{window}_{inst}.pick', "wb"))

In [9]:
def get_pos_nut(inst, t, nutrient):
    plate_num = plate_number[inst]
    side_P = get_Pside(plate_num).values[0]
    if nutrient == 'P':
        side_nut = side_P
    else:
        side_nut ='left' if side_P=='right' else 'left'
    if side_nut == 'none':
        side_nut = 'right'
    pos_nut = bait_positions[inst][t][side_nut]
    return(pos_nut,side_nut)

In [87]:
get_pos_nut((35,0,15),0,'P')

b


((32925, 13085), 'right')

In [34]:
def get_angle(veca,vecb):
    begin = veca/np.linalg.norm(veca)
    end = vecb/np.linalg.norm(vecb)
    dot_product = min(np.dot(begin, end),1)
    if (begin[1] * end[0] - end[1] * begin[0] >= 0):  # determinant
        angle = -np.arccos(dot_product) / (2 * np.pi) * 360
    else:
        angle = np.arccos(dot_product) / (2 * np.pi) * 360
    return(angle)

def get_curvature_density_bait(window, path,pos_baits):
    column_names = ["plate","inst", "treatment", "angle", "curvature","density","growth","speed",
                    "straightness","dist_P","dist_N","angle_to_P","angle_to_N","t","hyph"]
    infos = pd.DataFrame(columns=column_names)
    for treatment in ['25','baits']:
        insts = treatments[treatment]
        for inst in insts:
            exp = get_exp(inst,path)
            skeletons = [sparse.csr_matrix(skel) for skel in exp.skeletons]
            RH, BAS, max_speeds, total_growths, widths_sp, lengths, branch_frequ,select_hyph = get_rh_bas(exp,criter)
            pos_baits = bait_positions[inst]
            for hyph in RH:
                for i,t in enumerate(hyph.ts[:-1]):
                    pos_P, side_P = get_pos_nut(inst,t,'P')
                    pos_N, side_N = get_pos_nut(inst,t,'N')
                    tp1=hyph.ts[i+1]
                    segs,nodes = get_pixel_growth_and_new_children(hyph,t,tp1)
                    speed = np.sum([get_length_um(seg) for seg in segs])/get_time(exp,t,tp1)
                    total_growth = speed * get_time(exp,t,tp1)
                    curve = [pixel for seg in segs for pixel in seg]
                    pos = hyph.end.pos(t)
                    dist_P = np.linalg.norm(np.array(pos)-np.array(pos_P))
                    dist_N = np.linalg.norm(np.array(pos)-np.array(pos_N))
                    x,y = pos[0],pos[1]
                    straight_distance = np.linalg.norm(hyph.end.pos(t)-hyph.end.pos(tp1))
                    skeleton=skeletons[t][x-window:x+window,y-window:y+window]
                    density = skeleton.count_nonzero()/(window*1.725)
                    curve_array = np.array(curve)
                    res = 50
                    index = min(res,curve_array.shape[0]-1)
                    vec1 = curve_array [index]-curve_array [0]
                    vec2 = curve_array [-1]-curve_array [-index-1]
                    if np.linalg.norm(vec1)>0 and np.linalg.norm(vec2)>0:
                        angle = get_angle(vec1,vec2)
                        vec_growth =  (vec1+vec2)/2
                        vec_to_P = np.array(pos_P)-np.array(pos)
                        angle_to_P = get_angle(vec_growth,vec_to_P)
                        vec_to_N = np.array(pos_N)-np.array(pos)
                        angle_to_N = get_angle(vec_growth,vec_to_N)
                        inv_tortuosity = (straight_distance*1.725)/total_growth
                        if hyph.end.degree(tp1)<3 and inv_tortuosity>0.8 and inv_tortuosity<1.1:
                            if np.isnan((angle/total_growth)):
                                print(angle,total_growth,dot_product)
                            new_line = pd.DataFrame(
                                {   "plate": [plate_number[inst]],
                                    "inst": [inst],
                                    "treatment": [treatment],
                                    "angle": [angle],
                                    "curvature": [angle/total_growth],
                                    "density": [density],
                                    "growth": [total_growth],
                                    "speed": [speed],
                                    "straightness": [inv_tortuosity],
                                     "t": [get_time(exp,0,t)],
                                     "hyph": [hyph.end.label],
                                    "dist_P":[dist_P],
                                     "dist_N":[dist_N],
                                    "angle_to_P":[angle_to_P],
                                    "angle_to_N":[angle_to_N]
                                }
                            )  # index 0 for
                            # mothers need to be modified to resolve multi mother issue
                            infos = infos.append(new_line, ignore_index=True)
            break
        break
    return(infos)

In [35]:
infos = get_curvature_density_bait(800, directory_project,bait_positions)

begin = 2020-08-26 18:02:00 
  end = 2020-08-29 06:03:00


In [36]:
infos

,plate,inst,treatment,angle,curvature,density,growth,speed,straightness,dist_P,dist_N,angle_to_P,angle_to_N,t,hyph
0,351,"(35, 0, 15)",25,-44.627955,-0.220995,1.423188,201.941059,50.485265,0.908600,30174.769676,2555.543973,-166.513522,-171.246818,0.000000,0
1,351,"(35, 0, 15)",25,18.091350,0.063647,0.925362,284.245706,71.061426,0.956046,29408.835169,3883.177307,178.189595,118.890568,0.000000,18
2,351,"(35, 0, 15)",25,59.052703,0.218164,0.765217,270.680056,67.670014,0.910669,29627.815731,3950.743221,126.875173,69.009309,4.000000,18
3,351,"(35, 0, 15)",25,-10.236040,-0.040914,0.850000,250.186905,62.546726,0.994030,29827.664022,3954.542957,171.968238,118.122538,12.000000,18
4,351,"(35, 0, 15)",25,10.731002,0.033386,0.844203,321.420439,80.355110,0.988017,29968.457318,4013.256159,159.874112,107.556851,16.000000,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,351,"(35, 0, 15)",25,16.831669,0.033944,2.695652,495.858547,123.450261,0.993517,30123.640036,5507.035954,69.279857,10.448132,52.000000,18328
198,351,"(35, 0, 15)",25,11.477627,0.024167,2.403623,474.929134,118.732283,0.989865,30008.752490,5235.354525,63.271028,4.516098,56.016667,18328
199,351,"(35, 0, 15)",25,-1.287329,-0.003983,2.012319,323.236513,80.473821,0.997782,31278.590841,15190.895464,160.987925,98.778468,52.000000,18430
200,351,"(35, 0, 15)",25,4.020104,0.010164,2.113043,395.533246,98.883311,0.999591,31436.568785,15224.193443,168.497144,106.884749,56.016667,18430


In [26]:
window = 800

In [27]:
pickle.dump(infos, open(f'{path_code}/MscThesis/Results/straight_bait_{window}_{inst}.pick', "wb"))

In [30]:
infos2 = pickle.load(open(f'{path_code}/MscThesis/Results/straight_bait_{window}_{inst}.pick', "rb"))

In [33]:
inst

(13, 116, 131)